In [1]:
import syft
import imp
imp.reload(syft)
import numpy as np
from syft import FloatTensor,SyftController
# connect -- rerun this block if you make changes to the unity back-end
#         -- the controller is not yet smart enough to retry connection
#         -- on its own
sc = SyftController(verbose=False)

input = sc.FloatTensor([[0,0,1],[0,1.0,1],[1,0,1],[1,1,1]], autograd=True)
target = sc.FloatTensor([[0],[0],[1],[1]]).autograd(True)

weights_0_1 = sc.rand(3,4).autograd(True) - 0.5
weights_1_2 = sc.rand(4,1).autograd(True) - 0.5

for i in range(100):
    layer_1 = input.mm(weights_0_1).sigmoid()
    layer_2 = layer_1.mm(weights_1_2).sigmoid()

    loss = (layer_2 - target) ** 2 # Mean Squared Error Loss

    loss.backward()

    weights_0_1 -= weights_0_1.grad().T() * 0.5
    weights_1_2 -= weights_1_2.grad().T() * 0.5

    print(loss.to_numpy().sum())

0.9995215
0.9933668
0.9814494


Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/syft.py", line 32, in __del__
  File "/Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/syft.py", line 360, in delete_tensor
  File "/Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/syft.py", line 338, in no_params_func
  File "/Users/atrask/anaconda/lib/python3.6/site-packages/syft-0.1.0-py3.6.egg/syft/syft.py", line 323, in params_func
  File "/Users/atrask/anaconda/lib/python3.6/site-packages/zmq/sugar/socket.py", line 457, in recv_string
    msg = self.recv(flags=flags)
  File "zmq/backend/cython/socket.pyx", line 693, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:7683)
  File "zmq/backend/cython/socket.pyx", line 727, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:7460)
  File "zmq/backend/cython/socket.pyx", line 145,

ValueError: invalid literal for int() with base 10: 'SyftController.processMessage: Command not found.'